In [1]:
import syft as sy
import numpy as np
from syft.core.adp.entity_list import EntityList
from time import time
import pyarrow.parquet as pq

Loaded constant2epsilon cache of size: (300000,)


In [2]:
%%time
df = pq.read_table("/home/ruchi/1B_rows_dataset_sample.parquet")

CPU times: user 1min 40s, sys: 1min 30s, total: 3min 11s
Wall time: 14.2 s


In [3]:
t0 = time()
impressions = df['impressions'].to_numpy()
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 23.89875841140747 seconds


In [4]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=70, max_val=2000, entities = data_subjects, ndept=True)
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

Time make Private Syft Tensor: 2.2745814323425293 seconds


In [5]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into jolly_gardner... done!


In [6]:
domain_node.privacy_budget

5.55

In [7]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

# Upgrade admins budget
content = {"user_id": 1, "budget": 9_999_999}
domain_node._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

domain_node.privacy_budget

9999999.0

In [13]:
%%time
name = "1B Tweets dataset"

domain_node.load_dataset(
    assets={"1B Tweets dataset": tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

Loading dataset... uploading...🚀                                                                                                                                             

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft
Uploading `1B Tweets dataset`: 100%|██████████████████████████████████| 7/7 [00:08<00:00,  1.24s/it]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
CPU times: user 39.9 s, sys: 10.4 s, total: 50.3 s
Wall time: 57.1 s


In [14]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],1B Tweets dataset,Tweets- 1B rows,"[""1B Tweets dataset""] ->",d1131a15-fa76-4ccb-9c58-00f7b9e31fe2


In [15]:
data = domain_node.datasets[-1]["1B Tweets dataset"]

In [16]:
data

<TensorPointer -> jolly_gardner:5b34b60416004573a1543ad942cb52bb>

In [17]:
sum_result = data.sum()

In [18]:
%%time
sum_result.block

CPU times: user 2.79 s, sys: 149 ms, total: 2.94 s
Wall time: 1min 40s


<TensorPointer -> jolly_gardner:2f7496c268b24fd398646804e812b8a7>

In [19]:
sum_result.exists

True

In [20]:
published_result = sum_result.publish(sigma=1e6)

In [21]:
%time
published_result.block

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 24.6 µs


<FloatPointer -> jolly_gardner:fffbfcbfcece40a0ae150e972d8a7b50>

In [22]:
published_result.exists

True

In [23]:
res = published_result.get(delete_obj=False)
print(res)

1.1128860199797396e+16


In [24]:
domain_node.privacy_budget

9999991.233783375

In [25]:
impressions.sum()

11128860198757298

In [27]:
11128860198757298/1e16

1.1128860198757298

In [28]:
1.1128860199797396e+16 - 11128860198757298)

1040098.0